In [1]:
import os
import math
import itertools
import datetime
CUR_DIR = os.getcwd()
DATA_DIR = CUR_DIR + "/../data/raw/"
FILE_NAME = "execs.email.linesnum.txt"
#WRITE_NAME = "day_execs.email.linesnum.txt"
WRITE_NAME = "week_execs.email.linesnum.txt"

In [2]:
f = open(DATA_DIR+FILE_NAME,'r')
g = open(DATA_DIR+WRITE_NAME, 'w')
oneline = f.readline()
while(oneline!=""):
    entry = oneline.split()
    curTime = datetime.datetime.fromtimestamp(int(entry[0])).strftime('%Y-%m-%d')
    #g.write(curTime+' '+ str(int(entry[0])/86400) + ' '+str(int(entry[1])+1)+' '+str(int(entry[2])+1)+'\n')
    g.write(curTime+' '+ str((int(entry[0])+345600)/604800) + ' '+str(int(entry[1])+1)+' '+str(int(entry[2])+1)+'\n')
    oneline = f.readline()
g.close()
f.close()

In [3]:
f = open(DATA_DIR+WRITE_NAME,'r')
oneline = f.readline()
indexList = [[],[],[]]
while(oneline!=""):
    entry = oneline.split()
    for i in range(3):
        indexList[i].append(int(entry[i+1]))
    oneline = f.readline()
f.close()
    

In [4]:
n = len(indexList[1])
m = max(indexList[1])
tempMap = [i+1 for i in range(m)]
ind = m-1
for i in range(n):
    tempVal = indexList[0][i]
    if tempMap[ind]!=tempVal:
        tempMap.append(tempVal)
        ind += 1
indexMap = {}
for i in range(len(tempMap)):
    indexMap[tempMap[i]] = i+1

In [5]:
def get_tensor_dic(textFile, n):
    tensorDic = {}
    f = open(textFile,'r')
    oneline = f.readline()
    while(oneline!=""):
        tempList = []
        entry = oneline.split()
        for i in range(n):
            tempList.append(indexMap[int(entry[i+1])])
        tempTuple = tuple(tempList)
        if tempTuple not in tensorDic:
            tensorDic[tempTuple] = 1
        else:
            tensorDic[tempTuple] += 1
        oneline = f.readline()
    return tensorDic
    f.close()

In [6]:
def get_data_dic(tensorDic):
    dataDic = {}
    
    flagDic = {}
    for item in tensorDic:
        flagDic[item] = False
        
    for item in tensorDic:
        if flagDic[item] is True:
            continue
        flagDic[item] = True
        ip = itertools.permutations(item)
        tSum = 0
        for ipItem in ip:
            if ipItem in tensorDic:
                tSum += tensorDic[ipItem]
                flagDic[ipItem] = True
            
        ip = itertools.permutations(item)
        for ipItem in ip:
            dataDic[ipItem] = tSum
    
    return dataDic

In [7]:
def normalize_data(dataDic):
    colDic = {}; flagDic = {}
    for item in dataDic:
        flagDic[item] = False
        key = item[1:]
        if key not in colDic:
            colDic[key] = [item[0]]
        else:
            colDic[key].append(item[0])
    
    for item in dataDic:
        if flagDic[item] is True:
            continue
        key = item[1:]; tSum = 0
        for firstInd in colDic[key]:
            tSum += dataDic[(firstInd,)+key]
        for firstInd in colDic[key]:
            tempKey = (firstInd,)+key
            dataDic[tempKey] = (dataDic[tempKey]+0.00)/tSum
            flagDic[tempKey] = True

In [8]:
tensorDic = get_tensor_dic(DATA_DIR+WRITE_NAME, 3)
dataDic = get_data_dic(tensorDic)

In [9]:
def print_file(textFile, dataDic, n):
    
    f = open(textFile,'w')
    f.write("# This is the enron (time, from, to) tensor from http://www.cis.jhu.edu/~parky/Enron \n")
    f.write("# The last column is the tensor value, and other columns are tensor indices\n")
    for item in dataDic:
        tempList = []
        for num in item:
            tempList.append(str(num))
        indStr = '  '.join(tempList)
        f.write(indStr + "  " + str(dataDic[item]) + "\n")
    f.close()

In [10]:
#print_file(DATA_DIR+"/../time_from_to.txt", dataDic, 3)
print_file(DATA_DIR+"/../timeweek_from_to.txt", dataDic, 3)

In [11]:
normalize_data(dataDic)
#print_file(DATA_DIR+"/../ntime_from_to.txt", dataDic, 3)
print_file(DATA_DIR+"/../ntimeweek_from_to.txt", dataDic, 3)

In [12]:
#f = open(DATA_DIR+"/../dic_time_from_to.txt",'w')
f = open(DATA_DIR+"/../dic_timeweek_from_to.txt",'w')
f.write("# This is the enron (time, from, to) tensor from http://www.cis.jhu.edu/~parky/Enron \n")
f.write("# Each line denotes a word and its corrsponding index\n")
for item in indexMap:
    f.write(str(item) + "  " + str(indexMap[item]) + "\n")
f.close()